Install  Required Packages


In [1]:
!pip install langchain
!pip install openai
!pip install pyPDF2
!pip install faiss-cpu
!pip install tiktoken


#** Import all required packages**

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch,Pinecone,Weaviate,FAISS

#** step 03: Setup Environment**

In [3]:
import os 
!pip install load_dotenv


In [4]:
from dotenv import load_dotenv
load_dotenv() 
KEY = os.getenv("OPENAI_API_KEY")

#** step 04 : Extracting text from PDF**

In [5]:
reader = PdfReader('../PDFReader/pdfs/LLM.pdf')
#print(reader.getDocumentInfo)

In [6]:
#Read data from pdf : it will goto each page , raw_text will contain all pages
raw_text =''
for i, page in enumerate(reader.pages) :
    text = page.extract_text()
    if text:
        raw_text += text

In [7]:
print(raw_text)

LLM Apps 
The Universalization of AI 
© 2023 Julio Colomer, AI Accelera Phases of Artificial Intelligence 
●Machine Learning. 
●Internet >>> Data Science, Big Data. 
●Internet + Neural Networks + GPUs >>> Deep Learning 
●Data, Images. 
●...
●Language. Transformers. 
○Generative AI. 
○Github CoPilot, ChatGPT, Midjourney. 
○LLM Applications: the beginning of AI universalization. 
© 2023 Julio Colomer, AI Accelera The Universalization of AI 
●Before the launch of chatGPT, AI applications focused on solutions based 
mainly on data (supervised learning), and some on images (computer vision): 
recommendation systems, predictive analysis, image classification, etc. 
●The launch of chatGPT in November 2022 highlighted the potential of 
language-based AI applications (LLM applications) in practically all activities, 
thus beginning what we might call the era of "the universalization of AI." 
© 2023 Julio Colomer, AI Accelera LLM Applications, at the Heart of the Revolution 
●Just as all human a

In [8]:
raw_text[:1]

'L'

In [9]:
#chunk size of 1000  token each and there is going to be an overlap of 200 tokens between the consecutive chunks
#first chunk 1000 characters long , next chunk will include last 200 characters from the first chunk
textSplitter = CharacterTextSplitter(
    separator ="\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function =len 
)

In [10]:
#now to convert text into chunks we will use text splitter
texts  = textSplitter.split_text(raw_text)

In [11]:
len(texts)

2

In [12]:
text[0]

'L'

In [13]:
text[1]

'L'

Download Embeddings from OpenAI


In [14]:
#for each of the chunks we need to compute corresponding embeddings,we will be using OpenAI text embeddings

embeddings = OpenAIEmbeddings()

#we will use FAISS vector store , it will take text chunks find corresponding embedding and that will be stored in Document Search  
docsearch = FAISS.from_texts(texts,embeddings)

c:\Users\admin\anaconda3\envs\testopenai\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [15]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(),chain_type='stuff')


c:\Users\admin\anaconda3\envs\testopenai\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [16]:
query = "When was chatGPT launched"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs,question=query)

c:\Users\admin\anaconda3\envs\testopenai\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' November 2022'

In [17]:
query = "who is sonal shinde"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs,question=query)

" I don't know. This context does not mention anyone named Sonal Shinde."

In [18]:
query = "Before launch of ChatGPT how was AI"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs,question=query)

' Before the launch of ChatGPT, AI focused primarily on data solutions, such as recommendation systems and predictive analysis, as well as some image solutions like computer vision.'

In [19]:
query = "what is future of LLM developer"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs,question=query)

' Based on the given context, it can be inferred that the future of LLM developer is bright as LLM applications are becoming increasingly important in the field of AI. LLM developers are expected to be in high demand as they have the potential to play a key role in the universalization of AI.'